<a href="https://colab.research.google.com/github/chenchihwang/SDS/blob/main/base_(llama%2C_small_lm2)_model_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

load both base LLM models, create generate text functions

In [ ]:
# Install necessary libraries
!pip install -q transformers accelerate torch sentencepiece

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set device (use GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load models with token authentication
def load_model_and_tokenizer(model_name, use_auth_token=False):
    print(f"Loading {model_name}...")

    # Load tokenizer with authentication if needed
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_auth_token=use_auth_token  # Pass token for gated models
    )

    # Load model with authentication if needed
    if device == "cuda":
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True,
            use_auth_token=use_auth_token  # Pass token for gated models
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            use_auth_token=use_auth_token
        )
        model = model.to(device)

    print(f"{model_name} loaded successfully!")
    return model, tokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
Using device: cuda


this bottom cell needs setup as meta llama access is required, and auth setup to verify that needs to be set up

In [ ]:
# Install huggingface_hub if not already installed
!pip install -q huggingface_hub

# Login to Hugging Face
from huggingface_hub import login
login()

In [ ]:

# Load Llama-3.2-1B-Instruct with authentication
llama_model_name = "meta-llama/Llama-3.2-1B-Instruct"
llama_model, llama_tokenizer = load_model_and_tokenizer(llama_model_name, use_auth_token=True)

# Modified generate_text functions to handle long inputs but limit output

def generate_text_llama(prompt, max_new_tokens=30, temperature=0.7, top_p=0.9):
    """Generate text using Llama-3.2-1B-Instruct with long inputs but limited output"""
    formatted_prompt = f"<|begin_of_text|><|user|>\n{prompt}<|end_of_text|>\n<|assistant|>"

    # Tokenize with truncation handling to fit within model's limits
    inputs = llama_tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=4000  # Reserve space for generation within 4096 context
    ).to(device)

    with torch.no_grad():
        outputs = llama_model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,  # Limit ONLY the output length
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=llama_tokenizer.eos_token_id
        )

    generated_text = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "<|assistant|>" in generated_text:
        response = generated_text.split("<|assistant|>")[1].strip()
    else:
        response = generated_text

    return response

Loading meta-llama/Llama-3.2-1B-Instruct...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

meta-llama/Llama-3.2-1B-Instruct loaded successfully!


In [ ]:

# Load SmolLM2-1.7B-Instruct (no authentication needed)
smollm_model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
smollm_model, smollm_tokenizer = load_model_and_tokenizer(smollm_model_name)

# Generate text function for SmolLM2-1.7B-Instruct
def generate_text_smollm(prompt, max_new_tokens=30, temperature=0.7, top_p=0.9):
    """
    Generate text using SmolLM2-1.7B-Instruct

    Args:
        prompt: The prompt text to generate from
        max_length: Maximum length of generated text
        temperature: Controls randomness (lower = more deterministic)
        top_p: Nucleus sampling parameter

    Returns:
        Generated text
    """
    # Format prompt for SmolLM2-Instruct
    formatted_prompt = f"<|user|>\n{prompt}\n<|assistant|>"

    # Tokenize the prompt
    inputs = smollm_tokenizer(formatted_prompt, return_tensors="pt").to(device)

    # Generate text
    with torch.no_grad():
        outputs = smollm_model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,  # Limit ONLY the output length
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=smollm_tokenizer.eos_token_id
        )

    # Decode the generated text
    generated_text = smollm_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "<|assistant|>" in generated_text:
        response = generated_text.split("<|assistant|>")[1].strip()
    else:
        response = generated_text

    return response

Loading HuggingFaceTB/SmolLM2-1.7B-Instruct...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


HuggingFaceTB/SmolLM2-1.7B-Instruct loaded successfully!


In [ ]:
# Load TinyLlama-1.1B-Chat
tinyllama_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tinyllama_model, tinyllama_tokenizer = load_model_and_tokenizer(tinyllama_model_name)

# Generate text function for TinyLlama-1.1B-Chat
def generate_text_tinyllama(prompt, max_new_tokens=30, temperature=0.7, top_p=0.9):
    """
    Generate text using TinyLlama-1.1B-Chat-v1.0

    Args:
        prompt: The prompt text to generate from
        max_length: Maximum length of generated text
        temperature: Controls randomness (lower = more deterministic)
        top_p: Nucleus sampling parameter

    Returns:
        Generated text
    """
    # TinyLlama-Chat uses a format similar to Llama-2-Chat
    formatted_prompt = f"<|system|>\nYou are a helpful assistant.\n<|user|>\n{prompt}\n<|assistant|>"

    # Tokenize the prompt
    inputs = tinyllama_tokenizer(formatted_prompt, return_tensors="pt").to(device)

    # Generate text
    with torch.no_grad():
        outputs = tinyllama_model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,  # Limit ONLY the output length
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tinyllama_tokenizer.eos_token_id
        )

    # Decode the generated text
    generated_text = tinyllama_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    if "<|assistant|>" in generated_text:
        response = generated_text.split("<|assistant|>")[1].strip()
    else:
        response = generated_text

    return response

Loading TinyLlama/TinyLlama-1.1B-Chat-v1.0...
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded successfully!


quick check to see all generate text functions work

In [ ]:
# Example usage
example_prompt = "Explain the concept of neural networks in simple terms."

print("\n--- Llama-3.2-1B-Instruct Response ---")
llama_response = generate_text_llama(example_prompt)
print(llama_response)

print("\n--- SmolLM2-1.7B-Instruct Response ---")
smollm_response = generate_text_smollm(example_prompt)
print(smollm_response)

print("\n--- TinyLlama-1.1B-Chat Response ---")
tinyllama_response = generate_text_tinyllama(example_prompt)
print(tinyllama_response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Llama-3.2-1B-Instruct Response ---
Neural networks are a type of computer system that's inspired by the structure and function of the human brain. They're designed to analyze and learn from data

--- SmolLM2-1.7B-Instruct Response ---
Neural networks are a type of machine learning model that's inspired by the structure and function of the human brain. They're made up of layers of interconnected nodes, or "neurons," that process and transmit information.

Imagine a network of brain cells that communicate with each other to understand and respond to the world around them. Neural networks do something similar, using layers of interconnected nodes to process and transmit information.

Each node in the network receives input from the previous layer, performs some calculation, and then sends the output to the next layer. This process happens repeatedly, with each layer learning and improving the information it receives.

Neural networks are incredibly powerful tools for learning and rec

begin testing framework. how this works is as follows we utilize the following data set that we load in, in the next cell. it has speech that is in both eng and japanese, (for now, truthfully we should have more langauges...)
 1. then we take an utterance from spoken dialog of one person in japanese or english (chosen randomly at 50/50)
 2. measure if the response given by the llm corresponds with that latest langauge
 3. append the prompt and the answer as context to the next prompt.

 note we limit the response size of the llm to something really short, we simply want to measure if it is responding correctly or not. also note our context window size will also be pretty short as this increases really quickly, we can just use the latest 6 utterances from each speaker.

 4. simply repeat steps 1-3 on new utterances.

In [ ]:
!git clone https://github.com/ku-nlp/speechBSD.git
!cd speechBSD
!wget https://lotus.kuee.kyoto-u.ac.jp/~sshimizu/data/speechBSD_wav_20221026.tar.gz
!tar zxvf speechBSD_wav_20221026.tar.gz

Streaming output truncated to the last 5000 lines.
wav/train/190329_E03_02_spk0_no1_en.wav
wav/train/190315_E012_18_spk1_no4_ja.wav
wav/train/190315_E002_10_spk0_no20_ja.wav
wav/train/190329_J03_14_spk0_no34_en.wav
wav/train/190315_E008_10_spk1_no25_ja.wav
wav/train/190315_E008_03_spk0_no1_ja.wav
wav/train/190315_E002_03_spk0_no37_en.wav
wav/train/190329_E05_06_spk1_no21_en.wav
wav/train/190329_J03_07_spk0_no23_ja.wav
wav/train/190329_E05_07_spk2_no31_ja.wav
wav/train/190329_E06_12_spk0_no1_ja.wav
wav/train/190329_J04_20_spk2_no37_ja.wav
wav/train/190315_E007_14_spk1_no19_en.wav
wav/train/190315_J012_02_spk1_no2_en.wav
wav/train/190315_J006_20_spk0_no8_en.wav
wav/train/190315_J004_09_spk1_no7_ja.wav
wav/train/190315_J012_02_spk1_no16_ja.wav
wav/train/190329_E01_06_spk0_no16_ja.wav
wav/train/190329_E04_10_spk0_no23_en.wav
wav/train/190315_J002_18_spk0_no21_en.wav
wav/train/190329_J02_11_spk1_no21_ja.wav
wav/train/190315_E010_20_spk1_no21_ja.wav
wav/train/190315_J007_06_spk2_no14_en.wav


In [ ]:
import json
import random
import re

In [ ]:
!pip install langdetect
import langdetect  # Make sure to install with: pip install langdetect

def detect_language(text):
    """Detect if text is English or Japanese"""
    try:
        lang = langdetect.detect(text)
        # Map language codes to our simplified version
        if lang == 'ja':
            return 'ja'
        else:
            return 'en'  # Default to English for other languages
    except:
        return 'en'  # Default to English if detection fails

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f3841554648a60689aabbaecb002351501c4d4831211e30cca5d9decc9ef95b7
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


note in the current setup there is no special prompt manipulation of the three lms:
- smollm2,
- llama 3.2 1b
- TinyLlama/TinyLlama-1.1B-Chat


In [ ]:

def run_language_experiment_context(file_path, num_samples=100):
    """
    Run language experiments on multiple models with conversation history.
    Tests if models respond in the same language as the input.

    Args:
        file_path: Path to the JSON file containing conversations
        num_samples: Number of samples to test
    """
    # Load the data
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON: {e}")
        return

    # Convert to list if it's a single conversation
    conversations = data if isinstance(data, list) else [data]
    print(f"Loaded {len(conversations)} conversations")

    # Create a results dictionary for each model
    results = {
        "llama": {"total_queries": 0, "matching_responses": 0},
        "smollm": {"total_queries": 0, "matching_responses": 0},
        "tinyllama": {"total_queries": 0, "matching_responses": 0}
    }

    # Process each conversation
    conversation_idx = 0
    while (results["llama"]["total_queries"] < num_samples) and (conversation_idx < len(conversations)):
        conversation = conversations[conversation_idx]
        if "conversation" not in conversation:
            conversation_idx += 1
            continue

        # Get utterances from this conversation
        utterances = conversation["conversation"]
        if not utterances:
            conversation_idx += 1
            continue

        # Initialize conversation history for each model
        conversation_history = {
            "llama": [],
            "smollm": [],
            "tinyllama": []
        }

        # Process each utterance in the conversation
        for utterance in utterances:
            # Randomly choose English or Japanese
            lang_choice = random.choice(["en", "ja"])
            sentence_key = f"{lang_choice}_sentence"

            if sentence_key not in utterance or not utterance[sentence_key]:
                continue

            input_text = utterance[sentence_key]

            # Test each model
            for model_name in ["llama", "smollm", "tinyllama"]:
                # Skip if we've already reached the sample target for this model
                if results[model_name]["total_queries"] >= num_samples:
                    continue

                # When building prompts, add instructions to keep responses short
                prompt = input_text
                if conversation_history[model_name]:
                    history_text = ""
                    for prev_input, prev_response in conversation_history[model_name]:
                        history_text += f"{prev_input}\n{prev_response}\n"
                    prompt = f"{history_text}{input_text}\n\nKeep your response very brief, no more than 20 words."

                # Generate response with the appropriate model
                if model_name == "llama":
                    # response = generate_text_llama(prompt, max_length=100)
                    response = generate_text_llama(prompt, max_new_tokens=30)
                elif model_name == "smollm":
                    # response = generate_text_smollm(prompt, max_length=100)
                    response = generate_text_smollm(prompt, max_new_tokens=30)
                else:  # tinyllama
                    # response = generate_text_tinyllama(prompt, max_length=100)
                    response = generate_text_tinyllama(prompt, max_new_tokens=30)

                # Detect language of the response
                input_lang = lang_choice
                response_lang = detect_language(response)

                # Record statistics
                results[model_name]["total_queries"] += 1
                is_match = (response_lang == input_lang)
                if is_match:
                    results[model_name]["matching_responses"] += 1

                # Log this interaction
                print(f"Model: {model_name} - Sample {results[model_name]['total_queries']}: {input_lang} -> {response_lang} = {'MATCH' if is_match else 'NO MATCH'}")
                print(f"Input: {input_text}")
                print(f"Response: {response}")
                print("---")

                # Update conversation history
                conversation_history[model_name].append([input_text, response])
                # Limit history to last 10 interactions (5 from each speaker)
                if len(conversation_history[model_name]) > 10:
                    conversation_history[model_name] = conversation_history[model_name][-10:]

            # Check if we've reached our sample target for all models
            if all(results[model]["total_queries"] >= num_samples for model in results):
                break

        # Move to next conversation if needed
        conversation_idx += 1

    # Print and return results for each model
    print("\n===== EXPERIMENT RESULTS =====")
    for model_name, stats in results.items():
        total = stats["total_queries"]
        matching = stats["matching_responses"]
        match_percentage = (matching / total * 100) if total > 0 else 0

        print(f"\nModel: {model_name.upper()}")
        print(f"Total queries: {total}")
        print(f"Matching language responses: {matching}")
        print(f"Match percentage: {match_percentage:.2f}%")

    return results

# Example usage
# file_path = "path/to/your/conversations.json"
# run_language_experiment_context(file_path, num_samples=50)

In [ ]:
file_path = "/content/speechBSD/txt/train.json"
results = run_language_experiment_context(file_path, num_samples=100)
print(results)

Loaded 670 conversations
Model: llama - Sample 1: ja -> en = NO MATCH
Input: はい、K社システム開発部です。
Response: I can't help you with that. Is there anything else I can assist you with?
---
Model: smollm - Sample 1: ja -> ja = MATCH
Input: はい、K社システム開発部です。
Response: そうです。また、様々なテストのチ
---
Model: tinyllama - Sample 1: ja -> ja = MATCH
Input: はい、K社システム開発部です。
Response: そうです。K社システム開発部のようなものです。
---
Model: llama - Sample 2: ja -> en = NO MATCH
Input: H社の高市と申します。
Response: are you a student or a professional in the field?
---
Model: smollm - Sample 2: ja -> ja = MATCH
Input: H社の高市と申します。
Response: はい、K社システム開発部です。
---
Model: tinyllama - Sample 2: ja -> ja = MATCH
Input: H社の高市と申します。
Response: ありがとうございます。

ありがとうございます。
---
Model: llama - Sample 3: en -> ja = NO MATCH
Input: Thank you as always.
Response: ### 1 です。
---
Model: smollm - Sample 3: en -> en = MATCH
Input: Thank you as always.
Response: Here is a breakdown of the conversation:

User: How do I get the email address of a user on K-Society?
---
Model:

this new novel approach retains the original context in much of the same language. it has the following new features:
1. Add a self-checking mechanism to your prompt:
2. uses language tokens
3. short, language-specific "anchor text" at the beginning of your prompt and at the end to create a strong language context

In [ ]:
def generate_with_combined_language_enforcement(model_func, prompt, target_lang, max_new_tokens=30):
    """
    Combines multiple strategies to ensure the model responds in the target language.

    Args:
        model_func: The model generation function to use
        prompt: The input prompt
        target_lang: Target language ('ja' or 'en')
        max_new_tokens: Maximum number of tokens to generate

    Returns:
        Generated response in the target language
    """
    # 1. Language-specific anchors
    anchors = {
        "ja": {
            "prefix": "以下は日本語で答えます。",
            "suffix": "続けて日本語で："
        },
        "en": {
            "prefix": "The following response is in English.",
            "suffix": "Continue in English:"
        }
    }

    # 2. Language tokens
    lang_tokens = {
        "ja": "<jp>",
        "en": "<en>"
    }

    # 3. Self-checking instructions
    lang_names = {"ja": "Japanese", "en": "English"}

    # 4. Brevity instructions in the appropriate language
    brevity_instructions = {
        "ja": "回答は 20 語以内で簡潔にしてください。",
        "en": "Keep your response very brief, no more than 20 words."
    }

    # Create language-specific checker instructions
    if target_lang == "ja":
        checker_instructions = f"""
        重要：あなたの回答は必ず日本語のみで行ってください。
        まず、どの言語で回答しようとしているか確認してください。
        もし日本語でない場合は、最初からやり直して日本語で回答してください。
        {brevity_instructions[target_lang]}
        """
    else:  # English
        checker_instructions = f"""
        IMPORTANT: Your response MUST be in {lang_names[target_lang]} only.
        First, check what language you're about to respond in.
        If it's not {lang_names[target_lang]}, restart and respond in {lang_names[target_lang]}.
        {brevity_instructions[target_lang]}
        """

    # Combine all approaches
    enhanced_prompt = (
        f"{lang_tokens[target_lang]} {anchors[target_lang]['prefix']}\n\n"
        f"{prompt}\n\n"
        f"{checker_instructions}\n"
        f"{anchors[target_lang]['suffix']} {lang_tokens[target_lang]}"
    )

    # Generate response with the enhanced prompt
    return model_func(enhanced_prompt, max_new_tokens=max_new_tokens)

In [ ]:
def run_language_experiment_context_with_language_enforcement(file_path, num_samples=50):
    """
    Run language experiments on multiple models with enhanced language enforcement.
    Tests if models respond in the same language as the input.

    Args:
        file_path: Path to the JSON file containing conversations
        num_samples: Number of samples to test
    """
    # Load the data
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error loading JSON: {e}")
        return

    # Convert to list if it's a single conversation
    conversations = data if isinstance(data, list) else [data]
    print(f"Loaded {len(conversations)} conversations")

    # Create a results dictionary for each model
    results = {
        "llama": {"total_queries": 0, "matching_responses": 0},
        "smollm": {"total_queries": 0, "matching_responses": 0},
        "tinyllama": {"total_queries": 0, "matching_responses": 0}
    }

    # Process each conversation
    conversation_idx = 0
    while (results["llama"]["total_queries"] < num_samples) and (conversation_idx < len(conversations)):
        conversation = conversations[conversation_idx]
        if "conversation" not in conversation:
            conversation_idx += 1
            continue

        # Get utterances from this conversation
        utterances = conversation["conversation"]
        if not utterances:
            conversation_idx += 1
            continue

        # Initialize conversation history for each model
        conversation_history = {
            "llama": [],
            "smollm": [],
            "tinyllama": []
        }

        # Process each utterance in the conversation
        for utterance in utterances:
            # Randomly choose English or Japanese
            lang_choice = random.choice(["en", "ja"])
            sentence_key = f"{lang_choice}_sentence"

            if sentence_key not in utterance or not utterance[sentence_key]:
                continue

            input_text = utterance[sentence_key]

            # Test each model
            for model_name in ["llama", "smollm", "tinyllama"]:
                # Skip if we've already reached the sample target for this model
                if results[model_name]["total_queries"] >= num_samples:
                    continue

                # Build prompt with conversation history
                prompt = input_text
                if conversation_history[model_name]:
                    history_text = ""
                    for prev_input, prev_response in conversation_history[model_name]:
                        history_text += f"{prev_input}\n{prev_response}\n"
                    prompt = f"{history_text}{input_text}"

                # Get appropriate model function
                if model_name == "llama":
                    model_func = generate_text_llama
                elif model_name == "smollm":
                    model_func = generate_text_smollm
                else:  # tinyllama
                    model_func = generate_text_tinyllama

                # Generate response with enhanced language enforcement
                response = generate_with_combined_language_enforcement(
                    model_func=model_func,
                    prompt=prompt,
                    target_lang=lang_choice,
                    max_new_tokens=30
                )

                # Detect language of the response
                input_lang = lang_choice
                response_lang = detect_language(response)

                # Record statistics
                results[model_name]["total_queries"] += 1
                is_match = (response_lang == input_lang)
                if is_match:
                    results[model_name]["matching_responses"] += 1

                # Log this interaction
                print(f"Model: {model_name} - Sample {results[model_name]['total_queries']}: {input_lang} -> {response_lang} = {'MATCH' if is_match else 'NO MATCH'}")
                print(f"Input: {input_text}")
                print(f"Response: {response}")
                print("---")

                # Update conversation history
                conversation_history[model_name].append([input_text, response])
                # Limit history to fewer interactions to prevent context overflow
                if len(conversation_history[model_name]) > 6:
                    conversation_history[model_name] = conversation_history[model_name][-6:]

            # Check if we've reached our sample target for all models
            if all(results[model]["total_queries"] >= num_samples for model in results):
                break

        # Move to next conversation if needed
        conversation_idx += 1

    # Print and return results for each model
    print("\n===== EXPERIMENT RESULTS =====")
    for model_name, stats in results.items():
        total = stats["total_queries"]
        matching = stats["matching_responses"]
        match_percentage = (matching / total * 100) if total > 0 else 0

        print(f"\nModel: {model_name.upper()}")
        print(f"Total queries: {total}")
        print(f"Matching language responses: {matching}")
        print(f"Match percentage: {match_percentage:.2f}%")

    return results

In [ ]:
file_path = "/content/speechBSD/txt/train.json"
results = run_language_experiment_context_with_language_enforcement(file_path, num_samples=50)
print(results)

Loaded 670 conversations
Model: llama - Sample 1: en -> en = MATCH
Input: Hi this is the systems development department of Company K.
Response: I don't have a specific job opening to apply for at Company K. Can I provide a general inquiry about the company and its services?
---
Model: smollm - Sample 1: en -> en = MATCH
Input: Hi this is the systems development department of Company K.
Response: <en>
<|user|> I'm sorry, I didn't catch that. Could you please repeat what you said?
<en
---
Model: tinyllama - Sample 1: en -> en = MATCH
Input: Hi this is the systems development department of Company K.
Response: As the Systems Development Department of Company K, I am pleased to inform you that your response is in English only. Please ensure that your response meets
---
Model: llama - Sample 2: ja -> ja = MATCH
Input: H社の高市と申します。
Response: お気づきました。 Company K のシステム開発部門には、システム開発、エンジニアリング、データサイエンスなど、
---
Model: smollm - Sample 2: ja -> ja = MATCH
Input: H社の高市と申します。
Response: はい、先生。これは、このサイトのサー